# Get Search Results Using Zillow API 

### Overview

| Detail | Information | 
| --- | --- | 
| Input Data Source | Zillow API through Rapid API |
| Output Data Source | Pandas DataFrame |
| Future Developments | Output to DB, Database Modelling & Web Application|
| External References | Rapid API, Zillow API, and Ariel Herrera |

### History
| Date | Details | 
| --- | --- | 
| 24 April 2021 | Search results from Zillow API |

In [1]:
# load libraries
import pandas as pd 
import requests
from bs4 import BeautifulSoup

# show all cols in display
pd.options.display.max_columns = None

In [2]:
api = pd.read_csv('.../API.csv')
zwsid = api.loc[api['API'] == 'zillow']['KEY'].iloc[0]
rapid_api_key = api.loc[api['API'] == 'rapid']['KEY'].iloc[0]

Determine street, city, state and zipcode variable:

In [3]:
# address API parameters
street = '577 E 13th Ave'
city = 'Salt Lake City'
state = 'UT'
zipcode = '84103'

# street_formatted = street.replace(" ", "%20") #URL space encoding
# street_formatted

# city_formatted = city.replace(" ", "%20") 
# city_formatted

In [4]:
def search_results(zwsid, 
                   rapid_api_key,
                   street,
                   city,
                   state,
                   zipcode,
                   zestimate = True,
                   rent_zestimate = True):
    """
    Find a property for a specified address. 
    
    The content results contains the address for the property, zillow property id (ZPID) 
    and current Zestimate, date when Zestimate was computed, a valuation range, Zestimate
    ranking for the property within its zipcode. 
    
    The GetSearchResults API Web Service is located at: 
    https://www.zillow.com/howto/api/GetSearchResults.htm
    
    Parameters
    ----------
    @zwsid [string]: Zillow web service identified to access data from Zillow API
    @rapid_api_key [string]: Key to access data from Rapid API
    @street [string]: The street of the property
    @city [string]: The city of the property
    @state [string]: The state of the property
    @zipcode [string]: The zipcode of the property
    @zestimate [boolean]: Return Rent Zestimate information if available
    
    Returns
    -------
    [json] API response
    """
    # adjust params for paylod
    street_formatted = street.replace(" ", "%20") #URL space encoding
    city_formatted = city.replace(" ", "%20") # URL space encoding 
    if zestimate == True:
        zestimate_str = "true"
    else:
        zestimate_str = "false"
    if rent_zestimate == True:
        rent_zestimate_str = "true"
    else:
        rent_zestimate_str = "false"
    
    # url for api 
    url = "https://zillowdimashirokovv1.p.rapidapi.com/GetSearchResults.htm"

    # payload to send parameters 
    payload = "citystatezip={1}%20{2}%20{3}&zws-id={0}&rentzestimate=%5B{5}%2C{6}%5D&address={4}".format(zwsid,
                       city_formatted,
                       state,
                       str(zipcode),
                       street_formatted,
                       zestimate_str,
                       rent_zestimate_str)
    
    # header
    headers = {
    'content-type': "application/x-www-form-urlencoded",
    'x-rapidapi-key': rapid_api_key,
    'x-rapidapi-host': "ZillowdimashirokovV1.p.rapidapi.com"
    }

    # response
    response = requests.request("POST", url, data=payload, headers=headers)

    # check if there's an error on the request
    if response.status_code == 200: # no error
        return response.text
    else:
        return """Error {0}.See API source documentation for message and error
        code - https://www.zillow.com/howto/api/GetSearchResults.htm
        """.format(response.status_code)


In [5]:
def parse_search_results(response_content):
    """
    Parse search results to tabular format
    
    Parameters
    ----------
    @response_content [string]: Response text from GetSearchResults API
    
    Returns
    -------
    [dataframe] Tabular response
    """
    soup = BeautifulSoup(response_content, "xml")
    
    content_dict = {
        "street": [soup.find('street').text],
        "city": [soup.find('city').text],
        "state": [soup.find('state').text],
        "zipcode": [soup.find('zipcode').text],
        "zpid": [soup.find('zpid').text],
        "home_details": [soup.find('homedetails').text],
        #"graphsanddata": [soup.find('graphsanddata').text],
        "mapthishome": [soup.find('mapthishome').text],
        "comparables": [soup.find('comparables').text],
        "latitude": [soup.find("last-updated").text],
        "longitude": [soup.find('valueChange').text], 
        "zestimate_amount": [soup.find('amount').text],
        "zestimate_last_updated": [soup.find('last-updated').text],
        "zestimate_value_change": [soup.find('valueChange').text],
        "zestimate_range_low": [soup.find('valuationRange').find('low').text],
        "zestimate_range_high": [soup.find('valuationRange').find('high').text],
        "zillow_index_value": [soup.find('zindexValue').text]
        #"zillow_index_1YChange": [soup.find('zindexOneYearChange').text]
    }
    
    return pd.DataFrame.from_dict(content_dict)

In [6]:
# get api response
search_results_response = search_results(zwsid=zwsid,
                                rapid_api_key=rapid_api_key, 
                                street=street, 
                                city=city, 
                                state=state, 
                                zipcode=zipcode)

In [7]:
# transform xml response to df
df = parse_search_results(response_content = search_results_response)
df

,street,city,state,zipcode,zpid,home_details,mapthishome,comparables,latitude,longitude,zestimate_amount,zestimate_last_updated,zestimate_value_change,zestimate_range_low,zestimate_range_high,zillow_index_value
0,577 E 13th Ave,Salt Lake City,UT,84103,12717797,https://www.zillow.com/homedetails/577-E-13th-...,http://www.zillow.com/homes/12717797_zpid/,http://www.zillow.com/homes/comps/12717797_zpid/,04/24/2021,40502,1007069,04/24/2021,40502,926503,1097705,"418,700"


In [8]:
# get zestimate 
zestimate_value = df['zestimate_amount'].iloc[0]
zestimate_value

'1007069'

Get search results for multiple listings: 

In [9]:
addr = pd.read_csv('AddressParameters.csv', quotechar="'")
addr = addr.astype(str)
addr['street'] = addr['street'].replace(" ", "%20", regex=True)
addr['city'] = addr['city'].replace(" ", "%20", regex=True)
# print(addr.iloc[0][0])
# print(addr.iloc[0][1])
# print(addr.iloc[0][2])
# print(addr.iloc[0][3])

In [10]:
multiple_addr = []
for index, row in addr.iterrows():
    #print(row['street'], row['city'])
    search_results_response = search_results(zwsid=zwsid,
                                rapid_api_key=rapid_api_key, 
                                street=row['street'], 
                                city=row['city'], 
                                state=row['state'], 
                                zipcode=row['zipcode'])

    multiple_addr.append(search_results_response)

In [11]:
result = []
for x in range(len(multiple_addr)):
    #print(test[x])
    df = parse_search_results(response_content = multiple_addr[x])
    #print(df)
    result.append(df)

In [12]:
result[1]

,street,city,state,zipcode,zpid,home_details,mapthishome,comparables,latitude,longitude,zestimate_amount,zestimate_last_updated,zestimate_value_change,zestimate_range_low,zestimate_range_high,zillow_index_value
0,25 N 1000 W,Salt Lake City,UT,84116,12715445,https://www.zillow.com/homedetails/25-N-1000-W...,http://www.zillow.com/homes/12715445_zpid/,http://www.zillow.com/homes/comps/12715445_zpid/,04/24/2021,14896,280171,04/24/2021,14896,198921,310990,"158,700"


In [13]:
dff = pd.concat(result, ignore_index=True)

In [14]:
print(dff.head())

               street            city state zipcode       zpid  \
0  1023 W Learned Ave  Salt Lake City    UT   84116   12715444   
1         25 N 1000 W  Salt Lake City    UT   84116   12715445   
2         23 N 1000 W  Salt Lake City    UT   84116   12715446   
3         435 N 900 W  Salt Lake City    UT   84116   12715198   
4   821 W Fremont Ave  Salt lake city    UT   84104  305057101   

                                        home_details  \
0  https://www.zillow.com/homedetails/1023-W-Lear...   
1  https://www.zillow.com/homedetails/25-N-1000-W...   
2  https://www.zillow.com/homedetails/23-N-1000-W...   
3  https://www.zillow.com/homedetails/435-N-900-W...   
4  https://www.zillow.com/homedetails/821-W-Fremo...   

                                   mapthishome  \
0   http://www.zillow.com/homes/12715444_zpid/   
1   http://www.zillow.com/homes/12715445_zpid/   
2   http://www.zillow.com/homes/12715446_zpid/   
3   http://www.zillow.com/homes/12715198_zpid/   
4  http://www.zi

# END Notebook